# Combinatorial library data preparation

## Aim of this notebook

Extract information from the combinatorial library `json` file:

- Number of ligands
- Number of atoms for each recombined ligand
- Number of ligands that fulfill Lipinski's rule of five (Ro5)
- Number of ligands that fulfill the Ro5 criteria (i) molecular weight <= 500Da, (ii) number of hydrogen bond donors <= 5, (iii) number of hydrogen bond acceptors <= 10, and (iv) logP value <= 5 
- Number of ligands per subpocket combination
- Ligands with exact matches in original KLIFS ligands
- Ligands with substructure matches in original KLIFS ligands
- Ligands with exact matches in ChEMBL
- Most similar ligand in ChEMBL for each recombined ligand (molecule ChEMBL ID and similarity value)

Since the `json` file holds mulitple millions of ligands, we do this data processing once here at the beginning and save the results to separate files which will be used for analysis/visualization in the following notebooks.

## Table of contents

1. Combinatorial library data
2. Get properties from `json` file
3. Save properties to `csv`/`json` files

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from pathlib import Path

import ijson
import pandas as pd

## 1. Combinatorial library data

### Download data

The combinatorial library comes as large `json` file (5.6 GB).

**Note 1**: Due to its size, this file is not included in this GitHub repository but can be downloaded alongside all output files from this notebook from zenodo. 
Please follow the instructions given in `../data/combinatorial_library/README.md`.

**Note 2**: This notebook prepares data for the subsequent analysis notebooks. Since all output files from this notebook are included in the download, is it not necessary to rerun this notebook (takes about 20 minutes) to continue with the analysis notebooks. 

### Explain the data format

This `json` file contains a list of dictionaries, each describing a recombined ligand's properties:

`[ligand_1_dict, ligand_2_dict, ..., ligand_n_dict]`

This is an example dictionary:

```
{
    "bond_ids": [["AP_9", "SE_6"], ["GA_14", "AP_10"], ["B1_8", "GA_13"]], 
    "fragment_ids": ["B1_13", "SE_138", "GA_77", "AP_12"], 
    "hba": 1, 
    "hbd": 1, 
    "mwt": 1, 
    "logp": 1, 
    "n_atoms": 36, 
    "chembl_exact": 0, 
    "chembl_most_similar": ["CHEMBL4089123", 0.52], 
    "original_exact": 0, 
    "original_substructure": 0, 
    "inchi": "InChI=1S/C27H21FN6O2/c1-16-21(27(36)31-32(16)2)12-24(35)33-11-10-20-23(33)9-8-19(25(20)28)22-14-30-34-15-18(13-29-
26(22)34)17-6-4-3-5-7-17/h3-11,13-15H,12H2,1-2H3,(H,31,36)"
}
```

Each dictionary key contains the following value:

- `bond_ids` and `fragment_ids`: 
  - Bond IDs (`bond_ids`), e.g. `[["AP_9", "SE_6"], ["GA_14", "AP_10"], ["B1_8", "GA_13"]]`: Atom (`<subpocket>_<atom ID>`) pairs per fragment bond
  - Fragment IDs (`fragment_ids`), e.g. `["B1_13", "SE_138", "GA_77", "AP_12"]` (`<subpocket>_<fragment index in subpocket pool>`)
  - With this information it is possible to construct the recombined ligand from the fragment library
- `hba`, `hbd`, `mwt`, and `logp`: Ligand fulfills Lipinski's rule of five criteria? (`0` or `1`)
  - Number of hydrogen bond acceptors (`hba`) <= 10
  - Number of hydrogen bond donors (`hbd`) <= 5
  - Molecular weight (`mwt`) <= 500
  - LogP value (`logp`) <= 5 
- `n_atoms`: Number of heavy atoms
- `chembl_exact`: Ligand has exact match in ChEMBL? (`0` or `1`)
- `chembl_most_similar`: Most similar molecule in ChEMBL, e.g. `["CHEMBL4089123", 0.52]` (`[<molecule ChEMBL ID>, <Tanimoto similarity>]` 
- `original_exact`: Ligand has exact match in original ligands? (`0` or `1`)
- `original_substructure`: Ligand is substructure of original ligands? (`0` or `1`)
- `inchi`: InChI 

### Set file/folder paths

In [3]:
HERE = Path(_dh[-1])
PATH_FRAGMENT_LIBRARY = HERE / '../../data/fragment_library/'
PATH_COMBINATORIAL_LIBRARY = HERE / '../../data/combinatorial_library/combinatorial_library_deduplicated.json'

In order to access/filter the ligands' properties efficiently (time and memory), we use the `ijson` library:

> Ijson is an iterative JSON parser with standard Python iterator interfaces.

https://pypi.org/project/ijson/

## 2. Get properties from `json` file

In [4]:
def get_properties(path_combinatorial_library):
    """
    Get a set of properties from the combinatorial library json file:
    
    - Number of ligands
    - Number of atoms for each recombined ligand
    - Number of ligands that fulfill Lipinski's rule of five (Ro5)
    - Number of ligands that fulfill the Ro5 criteria (i) molecular weight <= 500Da, (ii) number of hydrogen bond donors <= 5, (iii) number of hydrogen bond acceptors <= 10, and (iv) logP value <= 5 
    - Number of ligands per subpocket combination
    - Ligands with exact matches in original KLIFS ligands
    - Ligands with substructure matches in original KLIFS ligands
    - Ligands with exact matches in ChEMBL
    - Most similar ligand in ChEMBL for each recombined ligand (molecule ChEMBL ID and similarity value)
    
    Parameters
    ----------
    path_combinatorial_library : pathlib.Path
        Path to combinatorial library json file.
    
    Returns
    -------
    dict
        Combinatorial library properties.
    """
    
    print(datetime.now())

    # Get object generator from json
    f = open(path_combinatorial_library, 'rb')
    objects = ijson.items(f, 'item')

    # Filter objects
    properties = {
        'n_ligands': 0,
        'n_atoms': [],
        'lipinski': 0,
        'mw': 0, 
        'logp': 0, 
        'hbd': 0, 
        'hba': 0, 
        'subpockets': {},
        'original_exact': [], 
        'original_substructure': [], 
        'chembl_exact': [],
        'chembl_most_similar': [],
        'chembl_highly_similar': []
    }
    
    for o in objects:
        
        # Get number of ligands and number of atoms per ligand
        properties['n_ligands'] += 1
        properties['n_atoms'].append(o['n_atoms'])
        properties['chembl_most_similar'].append(o['chembl_most_similar'])
        
        # Get number of subpocket combinations
        subpocket_key = "-".join(sorted([subpocket[:2] for subpocket in o['fragment_ids']]))
        if subpocket_key in properties['subpockets'].keys():
            properties['subpockets'][subpocket_key] +=1
        else:
            properties['subpockets'][subpocket_key] = 1
        
        # Get Lipinski's rule of five + criteria
        if o['mwt'] == 1:
            properties['mw'] += 1
        if o['logp'] == 1:
            properties['logp'] += 1
        if o['hbd'] == 1:
            properties['hbd'] += 1
        if o['hba'] == 1:
            properties['hba'] += 1
        if o['hba']+o['hbd']+o['mwt']+o['logp'] >= 3:
            properties['lipinski'] += 1
            
        # Get KLIFS and ChEMBL matches
        if o['original_exact'] == 1:
            properties['original_exact'].append(o)
        if o['original_substructure'] == 1:
            properties['original_substructure'].append(o)
        if o['chembl_exact'] == 1:
            properties['chembl_exact'].append(o)
        if o['chembl_most_similar'][1] is not None:
            if o['chembl_most_similar'][1] >= 0.90:
                properties['chembl_highly_similar'].append(o)

    properties['original_exact'] = pd.DataFrame(properties['original_exact'])
    properties['original_substructure'] = pd.DataFrame(properties['original_substructure'])
    properties['chembl_exact'] = pd.DataFrame(properties['chembl_exact'])
    properties['chembl_most_similar'] = pd.DataFrame(properties['chembl_most_similar'], columns=['chembl_id', 'similarity'])
    properties['chembl_highly_similar'] = pd.DataFrame(properties['chembl_highly_similar'])
        
    print(datetime.now())
    
    return properties

In [5]:
# Takes up to 30 minutes
properties = get_properties(PATH_COMBINATORIAL_LIBRARY)

2025-10-15 16:16:35.194828
2025-10-15 16:39:20.539334


## 3. Save properties to `csv`/`json` files

### Number of ligands

In [6]:
properties['n_ligands']
# NBVAL_CHECK_OUTPUT

11327471

### Number of atoms for each recombined ligand

In [7]:
# Takes a moment (20s)
pd.DataFrame(properties['n_atoms']).to_csv(
    '../../data/combinatorial_library/n_atoms.csv',
    index=None,
    header=None
)

### Number of ligands that fulfill Lipinski's rule of five (Ro5) + criteria

In [8]:
ro5 = pd.Series(
    {key: properties[key] for key in ['mw', 'logp', 'hbd', 'hba', 'lipinski', 'n_ligands']}
)
ro5
# NBVAL_CHECK_OUTPUT

mw            4576594
logp          6162558
hbd          11272327
hba           9738053
lipinski      6363664
n_ligands    11327471
dtype: int64

In [9]:
ro5.to_csv(
    '../../data/combinatorial_library/ro5.csv',
    header=None
)

### Number of ligands per subpocket combination

In [10]:
properties['subpockets']
# NBVAL_CHECK_OUTPUT

{'AP-FP-GA-SE': 9633673,
 'AP-B1-FP-GA': 183969,
 'AP-B2-FP-GA': 145120,
 'AP-B2-GA-SE': 211308,
 'AP-B1-GA-SE': 172871,
 'AP-B1-B2-GA': 2622,
 'AP-FP-SE': 729293,
 'AP-GA-SE': 110208,
 'AP-FP-GA': 127759,
 'AP-FP': 7037,
 'AP-GA': 617,
 'AP-SE': 1450,
 'AP-B2-GA': 876,
 'AP-B1-GA': 668}

In [11]:
subpockets = pd.DataFrame.from_dict(properties['subpockets'], orient="index")
subpockets = subpockets.rename(columns={0: 'count'})
subpockets.to_csv(
    '../../data/combinatorial_library/subpockets.csv'
)

### Ligands with exact matches in original KLIFS ligands

In [12]:
properties['original_exact'].shape[0]
# NBVAL_CHECK_OUTPUT

41

In [13]:
properties['original_exact'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[AP_7, FP_8], [SE_13, AP_8]]","[AP_15, FP_217, SE_1]",1,1,1,1,28,1,"[CHEMBL3971067, 1.0]",1,1,InChI=1S/C21H26N6O/c1-15(2)27-14-20(24-25-27)1...
1,"[[AP_9, FP_14]]","[AP_0, FP_30]",1,1,1,1,23,1,"[CHEMBL1789941, 1.0]",1,1,InChI=1S/C17H18N6/c18-7-5-15(12-3-1-2-4-12)23-...


In [14]:
properties['original_exact'].to_json(
    '../../data/combinatorial_library/original_exact.json'
)

### Ligands with substructure matches in original KLIFS ligands

In [15]:
properties['original_substructure'].shape[0]
# NBVAL_CHECK_OUTPUT

403

In [16]:
properties['original_substructure'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[AP_12, GA_3], [FP_6, AP_10], [SE_7, AP_11]]","[AP_12, FP_121, SE_27, GA_98]",1,1,1,1,26,0,"[CHEMBL399287, 0.91]",0,1,InChI=1S/C19H14N6S/c20-13-26-17-12-23-25-18(22...
1,"[[FP_6, AP_10], [AP_11, SE_6], [AP_12, GA_5]]","[AP_12, GA_27, SE_68, FP_52]",1,1,1,1,26,0,"[CHEMBL1643206, 0.84]",0,1,InChI=1S/C17H20N8S/c1-11-6-16(26-24-11)23-15-7...


In [17]:
properties['original_substructure'].to_json(
    '../../data/combinatorial_library/original_substructure.json'
)

### Ligands with exact matches in ChEMBL

In [18]:
properties['chembl_exact'].shape[0]
# NBVAL_CHECK_OUTPUT

385

In [19]:
properties['chembl_exact'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[AP_12, GA_3], [FP_6, AP_10], [SE_7, AP_11]]","[AP_12, FP_54, GA_12, SE_24]",1,1,1,1,26,1,"[CHEMBL4636757, 1.0]",0,0,InChI=1S/C20H27N5O/c1-4-16(13-26)23-18-10-19(2...
1,"[[AP_12, GA_4], [SE_7, AP_11], [FP_6, AP_10]]","[AP_12, GA_39, SE_26, FP_121]",1,1,1,1,27,1,"[CHEMBL399087, 1.0]",0,0,"InChI=1S/C19H14F3N5/c20-19(21,22)15-12-25-27-1..."


In [20]:
properties['chembl_exact'].to_json(
    '../../data/combinatorial_library/chembl_exact.json'
)

### Most similar ligand in ChEMBL for each recombined ligand

In [21]:
properties['chembl_most_similar'].shape[0]
# NBVAL_CHECK_OUTPUT

11327471

In [22]:
properties['chembl_most_similar'].head(2)

,chembl_id,similarity
0,CHEMBL1643262,0.52
1,CHEMBL4930052,0.5700000000000001


In [23]:
# Takes a moment (20s)
properties['chembl_most_similar'].to_json(
    '../../data/combinatorial_library/chembl_most_similar.json'
)

### Highly similar ligands in ChEMBL

In [24]:
properties['chembl_highly_similar'].shape[0]
# NBVAL_CHECK_OUTPUT

3783

In [25]:
properties['chembl_highly_similar'].head(2)
# NBVAL_CHECK_OUTPUT

,bond_ids,fragment_ids,hba,hbd,mwt,logp,n_atoms,chembl_exact,chembl_most_similar,original_exact,original_substructure,inchi
0,"[[AP_12, GA_6], [AP_11, SE_3], [FP_6, AP_10]]","[AP_12, SE_86, GA_21, FP_37]",1,1,1,1,25,0,"[CHEMBL4071527, 0.91]",0,0,InChI=1S/C19H16N6/c1-2-7-22-18-11-17(15-4-3-8-...
1,"[[AP_12, GA_3], [FP_6, AP_10], [SE_7, AP_11]]","[AP_12, FP_121, SE_26, GA_84]",1,1,1,1,26,0,"[CHEMBL249531, 0.98]",0,0,InChI=1S/C21H19N5/c1-2-7-18-15-24-26-20(23-14-...


In [26]:
properties['chembl_highly_similar'].to_json(
    '../../data/combinatorial_library/chembl_highly_similar.json'
)